Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  b1 = tf.Variable(tf.zeros([depth]))
  w2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  b2 = tf.Variable(tf.constant(1.0, shape=[depth]))
  w3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  b3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  w4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  b4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, w1, strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + b1)
    conv = tf.nn.conv2d(hidden, w2, strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + b2)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, w3) + b3)
    return tf.matmul(hidden, w4) + b4
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  print('-'*60)
  print('{:^12s}|{:^25s}|{:^12s}'.format("","Minibatch","Validation"))
  print('{:^12s}|{:^12s}|{:^12s}|{:^12s}'.format("Step","Loss","Accuracy","Accuracy"))
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('{:^12d}|{:^12.2f}|{:^12.2f}|{:^12.2f}'.format(step,l,accuracy(predictions, batch_labels),accuracy(valid_prediction.eval(), valid_labels)))
  print('-'*60)
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
------------------------------------------------------------
            |        Minibatch        | Validation 
    Step    |    Loss    |  Accuracy  |  Accuracy  
     0      |    5.18    |    6.25    |   10.00    
     50     |    1.84    |   37.50    |   38.92    
    100     |    0.73    |   81.25    |   63.65    
    150     |    1.10    |   68.75    |   72.80    
    200     |    1.14    |   68.75    |   77.72    
    250     |    0.79    |   68.75    |   75.09    
    300     |    0.28    |   93.75    |   78.61    
    350     |    1.10    |   56.25    |   78.96    
    400     |    0.58    |   87.50    |   80.20    
    450     |    0.30    |   87.50    |   80.55    
    500     |    0.37    |   87.50    |   79.83    
    550     |    0.52    |   81.25    |   81.03    
    600     |    0.31    |   93.75    |   80.92    
    650     |    0.53    |   93.75    |   80.67    
    700     |    0.66    |   68.75    |   81.25    
    750     |    0.89    |   75.00    |   8

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  b1 = tf.Variable(tf.zeros([depth]))
  w2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  b2 = tf.Variable(tf.constant(1.0, shape=[depth]))
  w3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  b3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  w4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  b4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, w1, strides=[1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(pool + b1)
    conv = tf.nn.conv2d(hidden, w2, strides=[1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(pool + b2)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, w3) + b3)
    return tf.matmul(hidden, w4) + b4
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  print('-'*60)
  print('{:^12s}|{:^25s}|{:^12s}'.format("","Minibatch","Validation"))
  print('{:^12s}|{:^12s}|{:^12s}|{:^12s}'.format("Step","Loss","Accuracy","Accuracy"))
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('{:^12d}|{:^12.2f}|{:^12.2f}|{:^12.2f}'.format(step,l,accuracy(predictions, batch_labels),accuracy(valid_prediction.eval(), valid_labels)))
  print('-'*60)
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
------------------------------------------------------------
            |        Minibatch        | Validation 
    Step    |    Loss    |  Accuracy  |  Accuracy  
     0      |    5.44    |    0.00    |   10.00    
     50     |    2.16    |   18.75    |   17.88    
    100     |    1.20    |   56.25    |   39.97    
    150     |    1.08    |   68.75    |   63.51    
    200     |    1.24    |   68.75    |   75.22    
    250     |    0.97    |   68.75    |   73.70    
    300     |    0.30    |   93.75    |   78.12    
    350     |    1.10    |   75.00    |   80.53    
    400     |    0.91    |   75.00    |   80.19    
    450     |    0.40    |   87.50    |   81.65    
    500     |    0.39    |   87.50    |   81.30    
    550     |    0.56    |   75.00    |   81.72    
    600     |    0.25    |   93.75    |   81.59    
    650     |    0.77    |   75.00    |   82.06    
    700     |    0.53    |   81.25    |   82.97    
    750     |    0.86    |   81.25    |   8

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [58]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 512
num_steps = 3000

keep_prob = 1
regularization_constant = 0.003
starting_learning_rate = 0.05

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  b1 = tf.Variable(tf.zeros([depth]))
  w2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  b2 = tf.Variable(tf.constant(1.0, shape=[depth]))
  w3 = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  b3 = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  w4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  b4 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, keep_prob = None):
    conv = tf.nn.conv2d(data, w1, strides=[1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(pool + b1)
    conv = tf.nn.conv2d(hidden, w2, strides=[1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(pool + b2)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    if keep_prob:
        w3_dropped = tf.nn.dropout(w3,keep_prob)
        w4_dropped = tf.nn.dropout(w4,keep_prob)
        hidden = tf.nn.relu(tf.matmul(reshape, w3_dropped) + b3)
        return (tf.matmul(hidden, w4_dropped) + b4),w3_dropped,w4_dropped
    else:
        hidden = tf.nn.relu(tf.matmul(reshape, w3) + b3)
        return tf.matmul(hidden, w4) + b4
  
  # Training computation.
  logits, w3_dropped, w4_dropped = model(tf_train_dataset, keep_prob)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss += regularization_constant * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) 
                                     + tf.nn.l2_loss(w3_dropped) + tf.nn.l2_loss(w4_dropped)) 
    
  # learning Rate
  global_step = tf.Variable(0., trainable=False)
  learning_rate = tf.train.exponential_decay(starting_learning_rate, global_step, 1000, 0.96, staircase=False)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  print('-'*60)
  print('{:^12s}|{:^25s}|{:^12s}'.format("","Minibatch","Validation"))
  print('{:^12s}|{:^12s}|{:^12s}|{:^12s}'.format("Step","Loss","Accuracy","Accuracy"))
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print('{:^12d}|{:^12.2f}|{:^12.2f}|{:^12.2f}'.format(step,l,accuracy(predictions, batch_labels),accuracy(valid_prediction.eval(), valid_labels)))
  print('-'*60)
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
------------------------------------------------------------
            |        Minibatch        | Validation 
    Step    |    Loss    |  Accuracy  |  Accuracy  
     0      |   11.61    |   18.75    |   10.00    
    100     |    5.06    |   87.50    |   72.18    
    200     |    5.58    |   75.00    |   78.77    
    300     |    4.63    |   93.75    |   80.41    
    400     |    4.82    |   81.25    |   81.59    
    500     |    4.58    |   87.50    |   81.97    
    600     |    4.25    |   93.75    |   81.66    
    700     |    4.72    |   81.25    |   82.96    
    800     |    4.26    |   81.25    |   83.12    
    900     |    4.13    |   81.25    |   83.07    
    1000    |    4.41    |   75.00    |   84.12    
    1100    |    4.07    |   75.00    |   83.44    
    1200    |    4.03    |   87.50    |   84.46    
    1300    |    3.79    |   87.50    |   84.48    
    1400    |    3.99    |   75.00    |   84.90    
    1500    |    4.22    |   62.50    |   8